# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [66]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
# Import API key
from api_keys import geoapify_key
import warnings
warnings.filterwarnings("ignore")

In [18]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,east london,-33.0153,27.9116,23.75,88,100,10.37,ZA,1737576115
1,1,kingston,17.9970,-76.7936,30.09,70,40,8.23,JM,1737575901
2,2,nizhneangarsk,55.7833,109.5500,-13.61,86,66,1.63,RU,1737576117
3,3,papatowai,-46.5619,169.4708,13.82,97,100,3.61,NZ,1737576118
4,4,san antonio de pale,-1.4014,5.6325,26.86,81,64,5.68,GQ,1737576120


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [28]:
# Configure the map plot
map_plt_1=city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 600,
    size = "Humidity",
    scale = 0.9,
    color = "City"

)
#Display the map
map_plt_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [55]:
# Narrow down cities that fit criteria and drop any results with null values
condition1= city_data_df[(city_data_df["Max Temp"]>21) & (city_data_df["Max Temp"]<27)]
condition2=condition1[condition1["Wind Speed"]<4.5]
narrow_data=condition2[condition2["Cloudiness"]==0]

# Drop any rows with null values
narrow_data_clean=narrow_data.dropna()

# Display sample data
narrow_data_clean

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
120,120,callao,-12.0667,-77.1500,24.16,69,0,4.12,PE,1737576265
259,259,goundam,16.4145,-3.6708,24.47,10,0,3.15,ML,1737576445
274,274,la dolorita,10.4883,-66.7861,26.91,52,0,1.34,VE,1737576463
360,360,dabhol,17.6000,73.1667,22.02,67,0,2.99,IN,1737576574
364,364,banamba,13.5500,-7.4500,25.32,15,0,3.20,ML,1737576579
394,394,timbuktu,16.7735,-3.0074,26.20,9,0,4.01,ML,1737576617
411,411,tejupilco de hidalgo,18.9000,-100.1500,25.84,32,0,2.63,MX,1737576579
437,437,mochudi,-24.4167,26.1500,23.70,37,0,1.93,BW,1737576670
510,510,saint-pierre,-21.3393,55.4781,25.82,69,0,1.54,RE,1737576770
522,522,gunjur,13.2019,-16.7339,22.04,37,0,3.60,GM,1737576786


### Step 3: Create a new DataFrame called `hotel_df`.

In [64]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=city_data_df[["City","Country","Lat","Lng","Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,east london,ZA,-33.0153,27.9116,88,
1,kingston,JM,17.9970,-76.7936,70,
2,nizhneangarsk,RU,55.7833,109.5500,86,
3,papatowai,NZ,-46.5619,169.4708,97,
4,san antonio de pale,GQ,-1.4014,5.6325,81,
...,...,...,...,...,...,...
536,olyokminsk,RU,60.3736,120.4264,96,
537,zalingei,SD,12.9096,23.4741,17,
538,mitsamiouli,KM,-11.3847,43.2844,85,
539,isangel,VU,-19.5500,169.2667,78,


,City,Country,Lat,Lng,Humidity,Hotel Name
43,saint-pierre,RE,-21.3393,55.4781,61,
66,badger,US,64.8000,-147.5333,37,
102,ust'-kulom,RU,61.6864,53.6902,43,
120,emerald,AU,-23.5333,148.1667,28,
130,newman,US,37.3138,-121.0208,51,
138,fort bragg,US,35.1390,-79.0060,80,
146,laguna,US,38.4210,-121.4238,52,
160,nova vicosa,BR,-17.8919,-39.3719,87,
170,fort myers shores,US,26.7092,-81.7459,95,
176,trinity beach,AU,-16.7888,145.6968,44,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [80]:
my_address="40800 N Apollo Way,Anthem,Arizona,United States of America"
params={
    "text":my_address,
    "apiKey":geoapify_key
}

base_url="https://api.geoapify.com/v1/geocode/search"

response = requests.get(base_url, params=params).json()

# Print the json (pretty printed)
print(json.dumps(response, indent=4, sort_keys=True))

{
    "error": "Unauthorized",
    "message": "Invalid apiKey",
    "statusCode": 401
}


In [6]:
# Set parameters to search for a hotel
radius = 10000
categories="accommodation.hotel"
params = {
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = # YOUR CODE HERE

    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saint-pierre - nearest hotel: Tropic Hotel
badger - nearest hotel: Clarion Hotel & Suites
ust'-kulom - nearest hotel: No hotel found
emerald - nearest hotel: Emerald Central Hotel
newman - nearest hotel: No hotel found
fort bragg - nearest hotel: Airborne Inn Lodging
laguna - nearest hotel: Holiday Inn Express & Suites
nova vicosa - nearest hotel: Pousada Pontal da Barra
fort myers shores - nearest hotel: Comfort Inn Fort Myers
trinity beach - nearest hotel: Blue Lagoon Resort
racine - nearest hotel: No hotel found
six-fours-les-plages - nearest hotel: Kyriad Direct La Seyne-sur-Mer
porto velho - nearest hotel: Hotel Slaviero Essential Porto Velho
posto fiscal rolim de moura - nearest hotel: No hotel found
port douglas - nearest hotel: Mantra Aqueous Port Douglas
roswell - nearest hotel: Best Western Plus Hotel
hell-ville - nearest hotel: ABUD HOTEL
sona - nearest hotel: Al Bosco
morehead city - nearest hotel: Bask Hotel Big Rock Landing, Trademark Collection by W

,City,Country,Lat,Lng,Humidity,Hotel Name
43,saint-pierre,RE,-21.3393,55.4781,61,Tropic Hotel
66,badger,US,64.8000,-147.5333,37,Clarion Hotel & Suites
102,ust'-kulom,RU,61.6864,53.6902,43,No hotel found
120,emerald,AU,-23.5333,148.1667,28,Emerald Central Hotel
130,newman,US,37.3138,-121.0208,51,No hotel found
138,fort bragg,US,35.1390,-79.0060,80,Airborne Inn Lodging
146,laguna,US,38.4210,-121.4238,52,Holiday Inn Express & Suites
160,nova vicosa,BR,-17.8919,-39.3719,87,Pousada Pontal da Barra
170,fort myers shores,US,26.7092,-81.7459,95,Comfort Inn Fort Myers
176,trinity beach,AU,-16.7888,145.6968,44,Blue Lagoon Resort


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)